# Using a miNDAR Package to Drive Exploration and Analysis

1. Connecting to a miNDAR containing Omics data from an NDAR Study
2. Querying data in the miNDAR
3. Exploring data in the miNDAR
4. Generating credentials for accessing s3 Objects
5. Using credentials and s3 Object references from the miNDAR to retreive data
6. Analyze data from an s3 Object

## 1. Connecting to a miNDAR containing Omics data from an NDAR Study

A Study Data Package can be generated by clicking the 'Download' button on a study from the [data from papers](https://ndar.nih.gov/data_from_papers.html).
Once packaged, these data can be pushed to a miNDAR in the form of where they will be availalbe as database tables and s3 object references.
When data has finished loading into a miNDAR, you will receive an email with the connection information.

- requires a Oracle client
- the following example uses [Oracle Instant Client](http://www.oracle.com/technetwork/topics/linuxx86-64soft-092277.html) v11.2 and the python library cx_Oracle

In [ ]:
import cx_Oracle
hostname = 'mindarvpc.cqahbwk3l1mb.us-east-1.rds.amazonaws.com'
port = 1521
sid = 'ORCL'

In [ ]:
import getpass
username = input('Enter your miNDAR username:')#'username_package_id'
password = getpass.getpass('Enter your miNDAR password:')
dsnStr = cx_Oracle.makedsn(hostname, port, sid )
db = cx_Oracle.connect(user=username, password=password, dsn=dsnStr)

## 2. Querying Data in the miNDAR

After negotiating a successful connection we can query the miNDAR directly using SQL.  We can retreive these results and store them as objects in R, Python, and many other programming languages.

Here we will query the table CONCEPT_BY_GUID, which is available in all miNDAR packages.

- in this example we store the results in a pandas dataframe (requries [pandas](http://pandas.pydata.org/))

In [ ]:
import pandas as pd
import pandas.io.sql as psql

c = db.cursor()
query="""SELECT 
           concept_id,
           subjectkey,
           interview_Age,
           gender, 
           short_name, 
           concept_hierarchy 
         FROM CONCEPT_BY_GUID"""
c.execute(query)
concept_by_guid = pd.DataFrame(c.fetchall())
concept_by_guid.columns = [rec[0] for rec in c.description]
c.close()
print(concept_by_guid)

Finally, here we are going to retreive data from the genomics_subject02 and genomcis_sample03 tables, which contain demographic and phenotypic data, and data on samples collected with file references for any data generated from these samples.

In [ ]:
query2 = """SELECT DISTINCT samp.SUBJECTKEY, subj.PHENOTYPE, samp.DATA_FILE1, samp.DATA_FILE2
           FROM GENOMICS_SAMPLE03 samp
           JOIN genomics_subject02 subj on samp.subjectkey=subj.subjectkey
           JOIN concept_by_guid conc on samp.subjectkey=conc.subjectkey
           WHERE conc.CONCEPT_HIERARCHY like '%IQ\\\\Low%'"""
c = db.cursor()
c.execute(query2)
data_files = pd.DataFrame(c.fetchall())
data_files.columns = [rec[0] for rec in c.description]

In [ ]:
print(data_files)

## 3. Generating credentials for accessing s3 Objects

Here we will generate credentials using our NIMH Data Archives username and password, which will allow us to authenticate and retreive data from AWS s3 Object storage for any shared objects.

- We will use a python package for generating tokens, which will be available on [GitHub](https://github.com/NDAR) after this webinar.

In [ ]:
import getpass
url = 'https://ndar.nih.gov/DataManager/dataManager'
username = input('Enter your NIMH Data Archives username:')
password = getpass.getpass('Enter your NIMH Data Archives password:')

In [ ]:
from nda_aws_token_generator import *
generator = NDATokenGenerator(url)
token = generator.generate_token(username, password)

In [ ]:
print('aws_access_key_id=%s\n'
      'aws_secret_access_key=%s\n'
      'security_token=%s\n'
      'expiration=%s\n' 
      %(token.access_key,
        token.secret_key,
        token.session,
        token.expiration)
      )

## 4. Using credentials and s3 Object references from the miNDAR to retreive data

Here we will combine our s3 credentials and the data we retrieved from the miNDAR, using the DATA_FILE2 column that has our s3 Object references to retreive some Variant Call data about a subject.

- We will use [boto](https://github.com/boto/boto) to interface with s3 objects 

In [ ]:
import boto.s3.connection
cf = boto.s3.connection.OrdinaryCallingFormat()
conn = boto.connect_s3( token.access_key, 
                        token.secret_key,
                        security_token=token.session,
                        calling_format=cf)

In [ ]:
from urllib.parse import urlparse
print(data_files.ix[1,'DATA_FILE2'])

In [ ]:
bucket = urlparse(data_files.ix[1,'DATA_FILE2']).netloc
key = urlparse(data_files.ix[1,'DATA_FILE2']).path
bucket_object = conn.get_bucket(bucket)
s3_object = boto.s3.key.Key(bucket_object)
s3_object.key = key
byte_data = s3_object.get_contents_as_string(headers={'Range': 'bytes=0-30000'})
#print(byte_data)

In [ ]:
print(byte_data.decode("utf-8"))

## 5. Analyze/Query data from an s3 Object

Here we will retreive multiple VCF files and search them for a specific gene of interest. This implementation uses a simple text search, which works for the annotated VCF files in this collection because SnpEff has been run, and the ID field contains a SNPEFF entry that includes gene name.

In [ ]:
from urllib.parse import urlparse
import time
import re
from colorama import init, Fore, Back
import re

def get_vcf(s3_ref):
    bucket = urlparse(s3_ref).netloc
    key = urlparse(s3_ref).path
    bucket_object = conn.get_bucket(bucket)
    s3_object = boto.s3.key.Key(bucket_object)
    s3_object.key = key
    byte_data = s3_object.get_contents_as_string()
    return byte_data.decode('utf-8')

gene_name = input('Enter the name of the gene: ')
for i in range(1,5): # Iterate over files 1-5; use data_files.index to iterate over all files in column.
    print('Streaming data from s3 Object [%s] to memory...' % data_files.ix[i, 'DATA_FILE2'])
    vcf_file = get_vcf(data_files.ix[i, 'DATA_FILE2'])
    print('Searching file contents for gene [%s]\r\n' % gene_name)
    count = 0
    for line in vcf_file.rstrip().split('\n'):
        if gene_name in line:
            count +=1
            init() # only necessary on Windows          
            print (re.sub(r'(.*)(' + re.escape(gene_name) +r')(.*)', r'\1' + Fore.MAGENTA + r'\2' + Fore.RESET + r'\3', line))        
    print('Found a total of [%s] variants annotated with this gene in the VCF file' % count)
    print('\r\n')

Here we will retreive multiple VCF files and use the PyVCF python package, returning results from specific region. This example demonstrates using the in-memory file stream as a file-like object with the PyVCF library to access specific information contianed in the VCF file.

In [ ]:
from urllib.parse import urlparse
import time
import re
from colorama import init, Fore, Back
import re
from io import StringIO
import vcf

def get_vcf(s3_ref):
    bucket = urlparse(s3_ref).netloc
    key = urlparse(s3_ref).path
    bucket_object = conn.get_bucket(bucket)
    s3_object = boto.s3.key.Key(bucket_object)
    s3_object.key = key
    byte_data = s3_object.get_contents_as_string()
    return byte_data.decode('utf-8')

print('Position-based query')
chrom = int(input('Enter the chromosome   : '))
start = int(input('Enter the Start Postion: '))
end   = int(input('Enter the End Position: '))
for i in range(1,5): # Iterate over files 1-5; use data_files.index to iterate over all files in column.
    print('Streaming data from s3 Object [%s] to memory...' % data_files.ix[i, 'DATA_FILE1'])
    vcf_file = StringIO(get_vcf(data_files.ix[i, 'DATA_FILE1']))
    vcf_reader = vcf.Reader(vcf_file)
    #vcf_reader.fetch(chrom, start, end)
    print('Matching records:')
    for record in vcf_reader:
        #print(record)
        if str(record.CHROM) == str(chrom) and (record.POS >= start and record.POS <= end):
            for sample in record.samples:
                call = record.genotype(sample.sample)
                if hasattr(call.data, 'PL'):
                    print('Sample: %s - %s DP=%s PL=%s QA=%s QR=%s RO=%s' % (sample, str(call.site),str(call.data.DP), str(call.data.PL), str(call.data.QA), str(call.data.QR), str(call.data.RO)))
                else:
                    print('Sample: %s - %s DP=%s QA=%s QR=%s RO=%s' % (sample, str(call.site),str(call.data.DP), str(call.data.QA), str(call.data.QR), str(call.data.RO)))
        

## 6. Query an indexed BAM file using samtools built with libcurl branch (supports authenticated access to secure AWS S3 objects)

First we connect to a miNDAR that has file references for BAM files.

In [ ]:
import getpass
username = input('Enter your miNDAR username:')#'username_packageid for miNDAR with BAM files'
password = getpass.getpass('Enter your miNDAR password:')
dsnStr = cx_Oracle.makedsn(hostname, port, sid )
db = cx_Oracle.connect(user=username, password=password, dsn=dsnStr)

In [ ]:
c = db.cursor()
c.execute(query2)
data_files = pd.DataFrame(c.fetchall())
data_files.columns = [rec[0] for rec in c.description]
print(data_files.ix[1,'DATA_FILE1'])
bam_file = data_files.ix[1, 'DATA_FILE1']

After negotiating our connection and retreiving a file reference, wefollow the instructions to build the libcurl branch of htslib and samtools.

https://www.biostars.org/p/147772/ -develop branch of samtools/htslib and samtools/samtools support access to private S3 objects

Then we set our environment variables using our AWS Federated Tokeb.

Finally we run samtools to view a specific region of Chromosome 20 of the indexed BAM file in s3. 

In [ ]:
#https://www.biostars.org/p/147772/ -develop branch of samtools/htslib and samtools/samtools support access to private S3 objects
#https://github.com/samtools/htslib/pull/232 Plan to merge this support into official 1.3 release of htslib/samtools
import os
os.environ['AWS_ACCESS_KEY_ID']=token.access_key
os.environ['AWS_SECRET_ACCESS_KEY']=token.secret_key
os.environ['AWS_SESSION_TOKEN']=security_token=token.session
!samtools view $bam_file 20:1000-100000 